In [1]:
from functools import reduce 
from pyspark.sql import DataFrame
from pyspark.sql import functions as f
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2 - Preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/02 10:56:09 WARN Utils: Your hostname, DESKTOP-EMCMOU9 resolves to a loopback address: 127.0.1.1; using 172.26.43.126 instead (on interface eth0)
22/09/02 10:56:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/02 10:56:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Transaction data
spark.read.parquet('data/tables/transactions_20210228_20210827_snapshot/order_datetime=2021-02-28')

user_id,merchant_abn,dollar_value,order_id
1,28000487688,133.22689421562643,0c37b3f7-c7f1-48c...
18485,62191208634,79.13140006851712,9e18b913-0465-4fd...
1,83690644458,30.441348317517228,40a2ff69-ea34-465...
18488,39649557865,962.8133405407585,f4c1a5ae-5b76-40d...
2,80779820715,48.12397733548124,cd09bdd6-f56d-489...
18489,43186523025,98.14878546968934,9008a98e-1b02-4de...
3,29566626791,46.33087226118639,26b7574e-81c2-455...
18490,93558142492,232.83335268750145,2bda0665-796f-4f2...
3,32361057556,87.34942171371054,633a7656-2fcc-4b8...
18491,64974914166,130.12601873970038,4bc15338-83eb-43d...


In [ ]:
# Merchant data
tbl_merchant = spark.read.parquet('data/tables/tbl_merchants.parquet')

name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162
Fusce Company,"[(gift, card, nov...",10206519221
Aliquam Enim Inco...,"[(computers, comP...",10255988167
Ipsum Primis Ltd,"[[watch, clock, a...",10264435225
Pede Ultrices Ind...,([computer progra...,10279061213
Nunc Inc.,"[(furniture, home...",10323485998


Clean `tags` column from merchant data.

In [ ]:
# Convert tags to lowercase, remove unwanted characters, and split text into array
tbl_merchants = tbl_merchants.withColumn('tags', f.lower(f.col('tags')))
tbl_merchants = tbl_merchants.withColumn('tags', f.regexp_replace('tags', r'[\(\)\[\]]', ''))
tbl_merchants = tbl_merchants.withColumn('tags', f.split(f.col('tags'), ','))

tbl_merchants.limit(10)

In [ ]:
# Consumer data
import pandas as pd
pd.read_csv("data/tables/tbl_consumer.csv", '|')

,name,address,state,postcode,gender,consumer_id
0,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1195503
1,Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
2,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female,154128
4,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female,712975
...,...,...,...,...,...,...
499994,Jessica Avila,508 Miranda Overpass Apt. 218,QLD,4400,Female,1385608
499995,Steven Thornton,7913 Schwartz Mission Suite 483,VIC,3097,Undisclosed,1466964
499996,Christy Smith,5681 Zachary Mountain Apt. 060,NSW,2756,Undisclosed,1253484
499997,Donna Sutton,54140 Jacob Point,VIC,3989,Female,175005


In [3]:
# Consumer mapping table
spark.read.parquet('data/tables/consumer_user_details.parquet')

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499
